In [2]:
import numpy as np
from quaos.symplectic import PauliSum, PauliString, Pauli, Xnd, Ynd, Znd, Id, string_to_symplectic, symplectic_to_string
from quaos.gates import GateOperation, Circuit, Hadamard as H, SUM as CX, PHASE as S
from quaos.hamiltonian import *
from quaos.core.prime_Functions_Andrew import int_to_bases
from collections import defaultdict
import sympy as sym
from sympy.physics.quantum import TensorProduct,Operator
#import math
import itertools
from itertools import combinations

In [ ]:
def xi(a, d):
    """
    Computes the a-th eigenvalue of a pauli with dimension d.
    
    Parameters:
        a (int): The integer to compute the eigenvalue for.
        d (int): The dimension of the pauli to use.
    
    Returns:
        complex: The computed eigenvalue.
    """
    return np.exp(2 * np.pi * 1j * a / d)

def group_indices(lst):
    """
    Groups indices of the same value in a list into sublists.
    For example, if the input list is [1, 2, 1, 3, 2], the output will be [[0, 2], [1, 4], [3]].
    """
    index_dict = defaultdict(list)
    for idx, value in enumerate(lst):
        index_dict[value].append(idx)
    
    return [indices for indices in index_dict.values()]


def Hadamard_Symmetric_PauliSum(n_paulis,n_qubits,n_sym_q):
    # create coefficients
    c_int_bands = np.sort(np.random.randint(n_paulis,size=n_paulis))
    c_bands = group_indices(c_int_bands)

    coefficients = np.zeros(n_paulis)
    sym_bands = []
    for i,b in enumerate(c_bands):
        coefficients[b] = np.random.normal(0, 1)
        if len(b) != 1:
            sym_bands.append(b)

    #print(sym_bands)
    n_extra_bands = np.floor(np.sum([len(b) for b in sym_bands])/2 - n_sym_q)
    pauli_strings = ['' for i in range(n_paulis)]

    all_x = []
    all_z = []
    for i in range(n_sym_q):
        x_pauli = []
        z_pauli = []
        if len(sym_bands) >= 1:
            b_ind = np.random.randint(len(sym_bands))
            b = sym_bands[b_ind]
            x_ind = np.random.randint(len(b))
            x_pauli.append(b[x_ind])
            b.pop(x_ind)
            z_ind = np.random.randint(len(b))
            z_pauli.append(b[z_ind])
            b.pop(z_ind)
            if len(b) < 2:
                sym_bands.pop(b_ind)
            else:
                sym_bands[b_ind] = b

            # randomly adding extra x and zs if possible
            if n_extra_bands > 0 and len(sym_bands) >= 1:
                extras = np.random.randint(n_extra_bands)
                n_extra_bands -= extras
                for j in range(extras):
                    b_ind = np.random.randint(len(sym_bands))
                    b = sym_bands[b_ind]
                    x_ind = np.random.randint(len(b))
                    x_pauli.append(b[x_ind])
                    b.pop(x_ind)
                    z_ind = np.random.randint(len(b))
                    z_pauli.append(b[z_ind])
                    b.pop(z_ind)
                    if len(b) < 2:
                        sym_bands.pop(b_ind)
                    else:
                        sym_bands[b_ind] = b
        print(coefficients[x_pauli])
        for j in range(n_paulis):
            if j in x_pauli:
                pauli_strings[j] += 'x1z0 '
            elif j in z_pauli:
                pauli_strings[j] += 'x0z1 '
            else:
                pauli_strings[j] += 'x0z0 '
        all_x += x_pauli
        all_z += z_pauli
    print(all_x,all_z)
    non_sym_paulis = [i for i in range(n_paulis) if not i in all_x and not i in all_z]
    q_dims = [2 for i in range(2*(n_qubits-n_sym_q))]
    available_paulis = list(np.arange(int(np.prod(q_dims))))
    for i,x in enumerate(all_x):
        pauli_index = np.random.choice(available_paulis)
        available_paulis.remove(pauli_index)
        exponents = int_to_bases(pauli_index, q_dims)
        for j in range(n_qubits-n_sym_q):
            r, s = int(exponents[2*j]), int(exponents[2*j+1])
            pauli_strings[x] += f"x{r}z{s} "
            pauli_strings[all_z[i]] += f"x{r}z{s} "

        pauli_strings[x].strip()
        pauli_strings[all_z[i]].strip()

    for i in non_sym_paulis:
        pauli_index = np.random.choice(available_paulis)
        available_paulis.remove(pauli_index)
        exponents = int_to_bases(pauli_index, q_dims)
        for j in range(n_qubits-n_sym_q):
            r, s = int(exponents[2*j]), int(exponents[2*j+1])
            pauli_strings[i] += f"x{r}z{s} "
        pauli_strings[i].strip()

    P = PauliSum(pauli_strings, weights=coefficients ,dimensions=[2 for i in range(n_qubits)], phases=None,standardise=False)
    #print(P)

    # construct random Clifford circuit
    C = Circuit(dimensions=[2 for i in range(n_qubits)])
    gate_list = [H,S,CX]
    gg = []
    for i in range(100):
        g_i = np.random.randint(3)
        if g_i == 2:
            aa = list(random.sample(range(n_qubits), 2))
            gg += [gate_list[g_i](aa[0],aa[1],2)]
            #print(aa)
        else:
            aa = list(random.sample(range(n_qubits), 1))
            gg += [gate_list[g_i](aa[0],2)]
        
    C.add_gate(gg)
    P = C.act(P)

    phases = P.phases
    cc = P.weights
    ss = P.pauli_strings
    dims = P.dimensions

    cc *= np.array([-1]*n_paulis)**phases
    P = PauliSum(ss, weights=cc ,dimensions=dims, phases=None,standardise=False)

    # qubits shuffle qubits (Fisher Yates Shuffle)
    '''
    gg = []
    order = np.arange(n_qubits)
    for i in np.arange(n_qubits)[::-1]:
        j = np.random.randint(i+1)
        gg += [GateOperation('SWAP',(i,j),['x1z0 x0z0 -> x0z0 x1z0', 'x0z1 x0z0 -> x0z0 x0z1', 'x0z0 x1z0 -> x1z0 x0z0', 'x0z0 x0z1 -> x0z1 x0z0'],2)]
        a = int(order[i])
        b = int(order[j])
        order[i] = b
        order[j] = a
        
    for g in gg:
        P = g.act(P)
    sym_qubit_ind = []
    for i in range(n_qubits):
        if order[i] in np.arange(n_sym_q):
            sym_qubit_ind.append(i)
    '''
    #print('Symmetric qubits: ',sym_qubit_ind)
    #print(P)
    return(P,C)

def add_s2(P,q0,q1,phase_mode):
    if phase_mode == 'SSSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())], 
                    gates=[CX(q0,q1,2),H(q1,2),S(q0,2),CX(q1,q0,2),H(q1,2),CX(q0,q1,2),S(q0,2)])
        return(C)
    elif phase_mode == 'DDSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),CX(q1,q0,2),H(q0,2),CX(q0,q1,2),H(q1,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'DSDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),H(q0,2),CX(q0,q1,2),CX(q1,q0,2),S(q0,2),CX(q1,q0,2),H(q1,2),CX(q0,q1,2),S(q0,2)])
        return(C)
    elif phase_mode == 'DSSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),H(q0,2),CX(q0,q1,2),S(q1,2),H(q1,2),CX(q1,q0,2),S(q0,2),CX(q1,q0,2),H(q1,2),CX(q0,q1,2),S(q0,2)])
        return(C)
    elif phase_mode == 'SDDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),H(q1,2),S(q1,2),CX(q1,q0,2),CX(q0,q1,2),S(q1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'SDSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q1,q0,2),S(q0,2),H(q1,2),CX(q0,q1,2),CX(q1,q0,2),S(q0,2)])
        return(C)    
    elif phase_mode == 'SSDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),H(q0,2),CX(q0,q1,2)])
        return(C) 
    elif phase_mode == 'DDDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),H(q0,2),S(q1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C) 

def add_r2(P,q0,q1,phase_mode):
    if phase_mode == 'SSSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())], 
                    gates=[S(q0,2),CX(q1,q0,2),S(q0,2)])
        return(C)
    elif phase_mode == 'DDSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q1,q0,2),H(q1,2),CX(q0,q1,2),CX(q1,q0,2),S(q0,2),CX(q1,q0,2),H(q1,2),CX(q0,q1,2),S(q0,2)])
        return(C)
    elif phase_mode == 'DSDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[S(q0,2),CX(q1,q0,2),S(q0,2),H(q1,2),CX(q0,q1,2),S(q1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'DSSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),S(q0,2),S(q1,2),CX(q0,q1,2),CX(q1,q0,2),S(q1,2),CX(q0,q1,2),S(q1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'SDDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[S(q0,2),CX(q1,q0,2),S(q0,2),S(q1,2),CX(q0,q1,2),S(q1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'SDSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q1,q0,2),H(q0,2),CX(q1,q0,2)])
        return(C)    
    elif phase_mode == 'SSDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q1,q0,2),H(q0,2),CX(q1,q0,2),CX(q0,q1,2),S(1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C) 
    elif phase_mode == 'DDDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),S(q0,2),S(q1,2),CX(q0,q1,2),CX(q1,q0,2)])
        return(C)  

def add_r2s2(P,q0,q1,phase_mode):
    if phase_mode == 'SSSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())], 
                    gates=[S(q0,2),CX(q1,q0,2),H(q1,2),CX(q1,q0,2),S(q0,2)])
        return(C)
    elif phase_mode == 'DDSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[S(q0,2),CX(q1,q0,2),CX(q0,q1,2),S(q0,2),H(q0,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'DSDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),S(q0,2),S(q1,2),H(q0,2),CX(q1,q0,2),S(q1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'DSSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[S(q0,2),CX(q1,q0,2),H(q1,2),CX(q1,q0,2),S(q0,2),S(q1,2),CX(q0,q1,2),S(q1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'SDDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),CX(q1,q0,2),S(q0,2),CX(q1,q0,2),H(q0,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'SDSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q1,q0,2),CX(q0,q1,2),S(q0,2),CX(q1,q0,2),H(q0,2),CX(q0,q1,2),S(q0,2)])
        return(C)    
    elif phase_mode == 'SSDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),S(q0,2),S(q1,2),H(q0,2),CX(q1,q0,2),CX(q0,q1,2)])
        return(C) 
    elif phase_mode == 'DDDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),CX(q1,q0,2),S(q0,2),H(q0,2),H(q1,2),S(q1,2),CX(q0,q1,2),S(q0,2)])
        return(C) 

def add_phase(P,cq,qubit,phase_mode):
    if phase_mode == 'SSSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())])
        return(C)
    elif phase_mode == 'DDSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[CX(cq,qubit,2),S(qubit,2),H(qubit,2),S(qubit,2),CX(cq,qubit,2)])
        return(C)
    elif phase_mode == 'DSDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[CX(cq,qubit,2),S(qubit,2),CX(cq,qubit,2),H(qubit,2),CX(cq,qubit,2),S(cq,2)])
        return(C)
    elif phase_mode == 'DSSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[S(qubit,2),CX(cq,qubit,2),S(qubit,2),H(qubit,2),S(qubit,2),CX(cq,qubit,2)])
        return(C)
    elif phase_mode == 'SDDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[S(qubit,2),H(qubit,2),CX(qubit,cq,2),S(cq,2),CX(qubit,cq,2),H(qubit,2),CX(cq,qubit,2),S(cq,2)])
        return(C)
    elif phase_mode == 'SDSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[CX(qubit,cq,2),S(cq,2),CX(qubit,cq,2),H(qubit,2),CX(cq,qubit,2),S(cq,2)])
        return(C)    
    elif phase_mode == 'SSDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[H(qubit,2),CX(qubit,cq,2),S(cq,2),CX(qubit,cq,2),H(qubit,2),CX(cq,qubit,2),S(cq,2)])
        return(C) 
    elif phase_mode == 'DDDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[CX(qubit,cq,2),S(cq,2),CX(qubit,cq,2),H(qubit,2),CX(cq,qubit,2),S(cq,2),CX(cq,qubit,2),S(qubit,2),H(qubit,2),S(qubit,2),CX(cq,qubit,2)])
        return(C)     

def number_of_I(P,Pauli_index,qubits):
    """
    Returns the number of I's in the Pauli string at Pauli_index for qubits.
    """

    return np.sum([1 for i in qubits if P.x_exp[Pauli_index,i] == 0 and P.z_exp[Pauli_index,i] == 0])

def flatten_list(nested_list):
    """
    Flattens a list of lists into a single list with all the elements.
    
    Args:
        nested_list (list of lists): The input list of lists.
    
    Returns:
        list: A single flattened list containing all elements.
    """
    return [item for sublist in nested_list for item in sublist]

def prepare_sym_candidates(P1,pi,pj,C,current_qubit):
    cq = current_qubit
    # prepare anti-commuting pauli strings with the same absolute coefficients for test of hadamard Symmetry
    # prime pauli pi and pj for cancel_pauli
    if P1.x_exp[pi, cq] == 1 and P1.z_exp[pj, cq] == 1: # x,z
        px = pi
        pz = pj
    elif P1.z_exp[pi, cq] == 1 and P1.x_exp[pj, cq] == 1: # z,x
        px = pj
        pz = pi
    elif P1.x_exp[pi, cq] == 1 and P1.z_exp[pj, cq] == 0: # x,id or x,x
        if any(P1.z_exp[pj, i] for i in range(cq,P1.n_qudits())):
            g = CX(cq,min([i for i in range(cq,P1.n_qudits()) if P1.z_exp[pj, i]]),2)
        elif any(P1.x_exp[pj, i] for i in range(cq,P1.n_qudits())):
            g = H(min([i for i in range(cq,P1.n_qudits()) if P1.x_exp[pj, i]]),2)
            P1 = g.act(P1)
            C.add_gate(g)
            g = CX(cq,min([i for i in range(cq,P1.n_qudits()) if P1.z_exp[pj, i]]),2)
        C.add_gate(g)
        P1 = g.act(P1)
        px = pi
        pz = pj
    elif P1.z_exp[pi, cq] == 1 and P1.x_exp[pj, cq] == 0: # z,id or z,z
        if any(P1.x_exp[pj, i] for i in range(cq,P1.n_qudits())):
            g = CX(min([i for i in range(cq,P1.n_qudits()) if P1.x_exp[pj, i]]),cq,2)
        elif any(P1.z_exp[pj, i] for i in range(cq,P1.n_qudits())):
            g = H(min([i for i in range(cq,P1.n_qudits()) if P1.z_exp[pj, i]]),2)
            P1 = g.act(P1)
            C.add_gate(g)
            g = CX(min([i for i in range(cq,P1.n_qudits()) if P1.x_exp[pj, i]]),cq,2)
        C.add_gate(g)
        P1 = g.act(P1)
        px = pj
        pz = pi
    elif P1.x_exp[pi, cq] == 0 and P1.z_exp[pj, cq] == 1: # id,z
        if any(P1.x_exp[pi, i] for i in range(cq,P1.n_qudits())):
            g = CX(min([i for i in range(cq,P1.n_qudits()) if P1.x_exp[pi, i]]),cq,2)
        elif any(P.z_exp[pi, i] for i in range(cq,P1.n_qudits())):
            g = H(min([i for i in range(cq,P1.n_qudits()) if P1.z_exp[pi, i]]),2)
            P1 = g.act(P1)
            C.add_gate(g)
            g = CX(min([i for i in range(cq,P1.n_qudits()) if P1.x_exp[pi, i]]),cq,2)
        C.add_gate(g)
        P1 = g.act(P1)
        px = pi
        pz = pj
    elif P1.x_exp[pi, cq] == 0 and P1.x_exp[pj, cq] == 1: # id,x
        if any(P1.z_exp[pi, i] for i in range(cq,P1.n_qudits())):
            g = CX(cq,min([i for i in range(cq,P1.n_qudits()) if P1.z_exp[pi, i]]),2)
        elif any(P1.x_exp[pi, i] for i in range(cq,P1.n_qudits())):
            g = H(min([i for i in range(cq,P1.n_qudits()) if P1.x_exp[pi, i]]),2)
            P1 = g.act(P1)
            C.add_gate(g)
            g = CX(cq,min([i for i in range(cq,P1.n_qudits()) if P1.z_exp[pi, i]]),2)
        C.add_gate(g)
        P1 = g.act(P1)
        px = pj
        pz = pi
    else: # id,id
        if any(P1.x_exp[pi, i] for i in range(cq,P1.n_qudits())):
            g = CX(min([i for i in range(cq,P1.n_qudits()) if P1.x_exp[pi, i]]),cq,2)
            P1 = g.act(P1)
            C.add_gate(g)
            if any(P1.z_exp[pj, i] for i in range(cq,P1.n_qudits())):
                g = CX(cq,min([i for i in range(cq,P1.n_qudits()) if P1.z_exp[pj, i]]),2)
            elif any(P1.x_exp[pj, i] for i in range(cq,P1.n_qudits())):
                g = H(min([i for i in range(cq,P1.n_qudits()) if P1.x_exp[pj, i]]),2)
                P1 = g.act(P1)
                C.add_gate(g)
                g = CX(cq,min([i for i in range(cq,P1.n_qudits()) if P1.z_exp[pj, i]]),2)
            C.add_gate(g)
            P1 = g.act(P1)
            px = pi
            pz = pj
        elif any(P1.z_exp[pi, i] for i in range(cq,P1.n_qudits())):
            g = CX(cq,min([i for i in range(cq,P1.n_qudits()) if P1.z_exp[pi, i]]),2)
            P1 = g.act(P1)
            C.add_gate(g)
            if any(P1.x_exp[pj, i] for i in range(cq,P1.n_qudits())):
                g = CX(min([i for i in range(cq,P1.n_qudits()) if P1.x_exp[pj, i]]),cq,2)
            elif any(P1.z_exp[pj, i] for i in range(cq,P1.n_qudits())):
                g = H(min([i for i in range(cq,P1.n_qudits()) if P1.z_exp[pj, i]]),2)
                P1 = g.act(P1)
                C.add_gate(g)
                g = CX(min([i for i in range(cq,P1.n_qudits()) if P1.x_exp[pj, i]]),cq,2)
            C.add_gate(g)
            P1 = g.act(P1)
            px = pj
            pz = pi
    return(C, P1, px, pz)

def check_current_paulis(P1,pi,pj,current_qubit,cc_bands):
    current_qubit_pauli_check = True
    for i in range(P1.n_paulis()):
        if i != pi and i != pj:
            if P1.x_exp[i, current_qubit] == 1 and P1.z_exp[i, current_qubit] == 1: # Y
                continue
            elif P1.x_exp[i, current_qubit] == 0 and P1.z_exp[i, current_qubit] == 0: # I
                continue
            elif (P1.x_exp[i, current_qubit] == 1 and P1.z_exp[i, current_qubit] == 0) or (P1.x_exp[i, current_qubit] == 0 and P1.z_exp[i, current_qubit] == 1): # X, Z
                if i not in flatten_list(cc_bands):
                    current_qubit_pauli_check = False
                    break
                else:
                    for b2 in cc_bands:
                        if i in b2:
                            if sum([P1.x_exp[j, current_qubit] for j in b2]) - sum([P1.z_exp[j, current_qubit] for j in b2]) == 0:
                                continue
                            else:
                                current_qubit_pauli_check = False
                                break
    return(current_qubit_pauli_check)

def circuit_copy(C):
    C2 = Circuit(dimensions=C.dimensions)
    for g in C.gates:
        C2.add_gate(g)
    return(C2)

def parity_difference(P,cq,q2,sym_pairs,pairs_parity):
    parity_dict = {'I':set([]),'X':set([]),'Z':set([]),'Y':set([])}
    for ip,pairs in enumerate(sym_pairs):
        if P.x_exp[pairs[0],q2] == 0 and P.z_exp[pairs[0],q2] == 0:
            if pairs_parity[ip] == 0:
                parity_dict['I'].add(0)
            else:
                parity_dict['I'].add(1)
        elif P.x_exp[pairs[0],q2] == 1 and P.z_exp[pairs[0],q2] == 0:
            if pairs_parity[ip] == 0:
                parity_dict['X'].add(0)
            else:
                parity_dict['X'].add(1)
        elif P.x_exp[pairs[0],q2] == 0 and P.z_exp[pairs[0],q2] == 1:
            if pairs_parity[ip] == 0:
                parity_dict['Z'].add(0)
            else:
                parity_dict['Z'].add(1)
        elif P.x_exp[pairs[0],q2] == 1 and P.z_exp[pairs[0],q2] == 1:
            if pairs_parity[ip] == 0:
                parity_dict['Y'].add(0)
            else:
                parity_dict['Y'].add(1)
    return(parity_dict)

def update_phase_parity(pairs_parity,phase_options,P2,current_qubit,sym_pairs):
    pairs_parity_copy = pairs_parity.copy()
    for k in range(len(phase_options)):
        #print(k)
        if phase_options[k] == 0: # SSSS
            pass
        elif phase_options[k] == 1: # DDSS
            for ip,pairs in enumerate(sym_pairs):
                if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    pass
            #
        elif phase_options[k] == 2: # DSDS
            for ip,pairs in enumerate(sym_pairs):
                if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    pass
            #
        elif phase_options[k] == 3: # DSSD
            for ip,pairs in enumerate(sym_pairs):
                if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
            #
        elif phase_options[k] == 4: # SDDS
            for ip,pairs in enumerate(sym_pairs):
                if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    pass
            #
        elif phase_options[k] == 5: # SDSD
            for ip,pairs in enumerate(sym_pairs):
                if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
            #
        elif phase_options[k] == 6: # SSDD
            for ip,pairs in enumerate(sym_pairs):
                if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
            #
        elif phase_options[k] == 7: # DDDD
            for ip,pairs in enumerate(sym_pairs):
                if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
    return(pairs_parity_copy)

def mode_key(n):
    if n == 0:
        return('SSSS')
    elif n == 1:
        return('DDSS')
    elif n == 2:
        return('DSDS')
    elif n == 3:
        return('DSSD')
    elif n == 4:
        return('SDDS')
    elif n == 5:
        return('SDSD')
    elif n == 6:
        return('SSDD')
    elif n == 7:
        return('DDDD')
    else:
        raise ValueError('Value has to be between 0 and 7')
    
def int_to_mode(n):
    if n == 0:
        return('SSSS')
    elif n == 1:
        return('DDSS')
    elif n == 2:
        return('DSDS')
    elif n == 3:
        return('DSSD')
    elif n == 4:
        return('SDDS')
    elif n == 5:
        return('SDSD')
    elif n == 6:
        return('SSDD')
    elif n == 7:
        return('DDDD')
    else:
        raise ValueError('Value has to be between 0 and 7')
    
def transform_list(lst):
    """
    Convert a list of 4 elements (0, 1, or 2) into a string where:
    - 0 → 'S'
    - 1 → 'D'
    - 2 → Either 'S' or 'D', chosen to ensure an even count of 'S' or 'D'
    
    Parameters:
        lst (list): A list of four elements containing 0s, 1s, and 2s.

    Returns:
        str: The resulting string with balanced 'S' and 'D'.
    """
    base_chars = ['S' if x == 0 else 'D' if x == 1 else None for x in lst]
    unknown_indices = [i for i, x in enumerate(lst) if x == 2]

    # Generate all possible assignments for '2's
    possible_values = ['S', 'D']
    for assignment in combinations(possible_values * len(unknown_indices), len(unknown_indices)):
        temp_chars = base_chars[:]
        for i, val in zip(unknown_indices, assignment):
            temp_chars[i] = val
        
        # Check if count of 'S' or 'D' is even
        if temp_chars.count('S') % 2 == 0 or temp_chars.count('D') % 2 == 0:
            return ''.join(temp_chars)
    
    return None  # In case no valid assignment is found (shouldn't happen)




def remove_Ys(P1,C,current_qubit,q_print = False):
    cq = current_qubit
    sym_paulis_candidates = [i for i in range(P1.n_paulis()) if (P1.x_exp[i,cq] == 1 and P1.z_exp[i,cq] == 0) or (P1.x_exp[i,cq] == 0 and P1.z_exp[i,cq] == 1)]
    if q_print:
        print('sym_paulis_candidates',sym_paulis_candidates)
    other_paulis = [i for i in range(P1.n_paulis()) if i not in sym_paulis_candidates]
    if q_print:
        print('other_paulis',other_paulis)
    P2 = P1.copy()

    C2 = Circuit(dimensions=C.dimensions)#circuit_copy(C)
    
    h_sym = False
    h_sym_impossible = False
    no_forcings = False
    no_qubits = False

    usable_qubits = [i for i in range(cq+1,P1.n_qudits())]
    gate_options = [[0,1,2,3] for i in range(cq+1,P1.n_qudits())]
    other_first_qubit_x = P2.x_exp[[j for j in other_paulis],cq] # Array of everything that is still Y or I in the current qubit
    all_first_qubit_x = P2.x_exp[[j for j in range(P2.n_paulis())],cq]
    if q_print:
        print('other_first_qubit_x',other_first_qubit_x)

    #print(gate_options)
    while not no_forcings and len(usable_qubits) > 0 and not h_sym and not h_sym_impossible:
        # For each PS, how many identities does it have on the still 'usable_qubits'
        list_n_I = [number_of_I(P2,Pauli_index,usable_qubits) for Pauli_index in range(P2.n_paulis())]
        if q_print:
            print('list_n_I',list_n_I)
        # Locate Paulis that have all Identities (except for one) in the usable qubits and are not candidates for symmetry
        forcing_candidates_paulis = [i for i in range(P2.n_paulis()) if list_n_I[i] == len(usable_qubits) - 1 and i not in sym_paulis_candidates]
        if q_print:
            print('forcing_candidates_paulis',forcing_candidates_paulis)
        # locate the qubits on which the individual non-I paulis are
        forcing_candidates_qubits = []
        for i in forcing_candidates_paulis:
            qubit_index = min([j for j in usable_qubits if (P2.x_exp[i,j]+P2.z_exp[i,j]) != 0])
            if qubit_index not in forcing_candidates_qubits:
                forcing_candidates_qubits.append(qubit_index)
        if q_print:
            print('forcing_candidates_qubits',forcing_candidates_qubits)
        # now go through the forcing candidates for the qubits and see if they actually force to use a certain gate
        no_forcings = True
        for qi in forcing_candidates_qubits:
            # select the PS that have there sole non-I pauli (in the usable qubits) on the current forcing candidate qubit
            pauli_indexes = [i for i in forcing_candidates_paulis if (P2.x_exp[i,qi]+P2.z_exp[i,qi]) != 0]
            if q_print:
                print('pauli_indexes',pauli_indexes)
            # for the currently selected PS, what are the Paulis in the first qubit
            first_qubit_x = all_first_qubit_x[[j for j in pauli_indexes]] #P2.x_exp[[j for j in pauli_indexes],cq]
            if q_print:
                print('first_qubit_x',first_qubit_x)
            
            # count how many different non-I Paulis are in the set of selected PS (need to be at least 2 for forcing)
            forcing_pauli_types = set([])
            for i in pauli_indexes:
                if P2.x_exp[i,qi] == 1 and P2.z_exp[i,qi] == 0:
                    forcing_pauli_types.add(1)
                elif P2.x_exp[i,qi] == 0 and P2.z_exp[i,qi] == 1:
                    forcing_pauli_types.add(2)
                elif P2.x_exp[i,qi] == 1 and P2.z_exp[i,qi] == 1:
                    forcing_pauli_types.add(3)
            if q_print:
                print('forcing_pauli_types',forcing_pauli_types)
            
            # now see if one of the 4 possibilities can turn the paulis in the current qubit for the selected PS into I's
            if len(pauli_indexes) > 1 and len(forcing_pauli_types) > 1:
                if not any(first_qubit_x):
                    # Identity
                    if 1 in gate_options[qi-(1+cq)]:
                        gate_options[qi-(1+cq)].remove(1)
                    if 2 in gate_options[qi-(1+cq)]:
                        gate_options[qi-(1+cq)].remove(2)
                    if 3 in gate_options[qi-(1+cq)]:
                        gate_options[qi-(1+cq)].remove(3)
                    usable_qubits.remove(qi)

                elif not any((first_qubit_x + P2.x_exp[[j for j in pauli_indexes],qi])%2):
                    # add_r2
                    if 0 in gate_options[qi-(1+cq)]:
                        gate_options[qi-(1+cq)].remove(0)
                    if 2 in gate_options[qi-(1+cq)]:
                        gate_options[qi-(1+cq)].remove(2)
                    if 3 in gate_options[qi-(1+cq)]:
                        gate_options[qi-(1+cq)].remove(3)
                    #g = add_r2(P1,cq,qi)
                    #for gg in g.gates:
                    #    C2.add_gate(gg)
                    #P2 = g.act(P2)
                    other_first_qubit_x = (other_first_qubit_x + P2.x_exp[other_paulis,qi])%2
                    all_first_qubit_x = (all_first_qubit_x + P2.x_exp[:,qi])%2
                    
                    usable_qubits.remove(qi)
                elif not any((first_qubit_x + P2.z_exp[[j for j in pauli_indexes],qi])%2):
                    # add_s2
                    if 0 in gate_options[qi-(1+cq)]:
                        gate_options[qi-(1+cq)].remove(0)
                    if 1 in gate_options[qi-(1+cq)]:
                        gate_options[qi-(1+cq)].remove(1)
                    if 3 in gate_options[qi-(1+cq)]:
                        gate_options[qi-(1+cq)].remove(3)
                    #g = add_s2(P1,cq,qi)
                    #P2 = g.act(P2)
                    #for gg in g.gates:
                    #    C2.add_gate(gg)
                    other_first_qubit_x = (other_first_qubit_x + P2.z_exp[other_paulis,qi])%2
                    all_first_qubit_x = (all_first_qubit_x + P2.z_exp[:,qi])%2
                    usable_qubits.remove(qi)
                elif not any((first_qubit_x + P2.z_exp[[j for j in pauli_indexes],qi] + P2.x_exp[[j for j in pauli_indexes],qi])%2):
                    # add_r2s2
                    if 0 in gate_options[qi-(1+cq)]:
                        gate_options[qi-(1+cq)].remove(0)
                    if 1 in gate_options[qi-(1+cq)]:
                        gate_options[qi-(1+cq)].remove(1)
                    if 2 in gate_options[qi-(1+cq)]:
                        gate_options[qi-(1+cq)].remove(2)
                    #g = add_r2s2(P1,cq,qi)
                    #P2 = g.act(P2)
                    #for gg in g.gates:
                    #    C2.add_gate(gg)
                    other_first_qubit_x = (other_first_qubit_x + P2.x_exp[other_paulis,qi] + P2.z_exp[other_paulis,qi])%2
                    all_first_qubit_x = (all_first_qubit_x + P2.z_exp[:,qi])%2
                    usable_qubits.remove(qi)
                else:
                    # Forcing Impossible
                    if 0 in gate_options[qi-(1+cq)]:
                        gate_options[qi-(1+cq)].remove(0)
                    if 1 in gate_options[qi-(1+cq)]:
                        gate_options[qi-(1+cq)].remove(1)
                    if 2 in gate_options[qi-(1+cq)]:
                        gate_options[qi-(1+cq)].remove(2)
                    if 3 in gate_options[qi-(1+cq)]:
                        gate_options[qi-(1+cq)].remove(3)
                    h_sym_impossible = True
                    usable_qubits.remove(qi)
                    return(P1,C)
                no_forcings = False
                break
            else:
                '''
                print('single Pauli forcing')
                print(gate_options)
                print(qi-(1+cq))
                print(first_qubit_x)
                if 0 in gate_options[qi-(1+cq)]:
                    if any(first_qubit_x):
                        gate_options[qi-(1+cq)].remove(0)
                print((first_qubit_x + P2.x_exp[[j for j in pauli_indexes],qi])%2)
                if 1 in gate_options[qi-(1+cq)]:
                    if any((first_qubit_x + P2.x_exp[[j for j in pauli_indexes],qi])%2):
                        gate_options[qi-(1+cq)].remove(1)
                print((first_qubit_x + P2.z_exp[[j for j in pauli_indexes],qi])%2)
                if 2 in gate_options[qi-(1+cq)]:
                    if any((first_qubit_x + P2.z_exp[[j for j in pauli_indexes],qi])%2):
                        gate_options[qi-(1+cq)].remove(2)
                print((first_qubit_x + P2.z_exp[[j for j in pauli_indexes],qi] + P2.x_exp[[j for j in pauli_indexes],qi])%2)
                if 3 in gate_options[qi-(1+cq)]:
                    if any((first_qubit_x + P2.z_exp[[j for j in pauli_indexes],qi] + P2.x_exp[[j for j in pauli_indexes],qi])%2):
                        gate_options[qi-(1+cq)].remove(3)
                '''
                continue
        first_qubit_x = P2.x_exp[other_paulis,cq]
        if q_print:
            print('usable_qubits',usable_qubits)
            print('gate_options',gate_options)
        if not any(first_qubit_x%2):
            h_sym = True
    
    if q_print:
        print('other_first_qubit_x',other_first_qubit_x)

    if q_print:
        print()
        print('h_sym',h_sym)
        print('h_sym_impossible',h_sym_impossible)
    if not h_sym and not h_sym_impossible:
        addition_options_lens = []

        for l in gate_options:
            addition_options_lens.append(len(l))
        if q_print:
            print('addition_options_lens',addition_options_lens)
        max_options = np.prod(addition_options_lens)

        for i in range(max_options):
            addition_options_temp = int_to_bases(i,dims=addition_options_lens[::-1])[::-1]
            addition_options = np.zeros(len(addition_options_temp))
            for j in range(len(addition_options_temp)):
                addition_options[j] = gate_options[j][addition_options_temp[j]]

            first_qubit_x = P2.x_exp[[j for j in other_paulis],cq]
            for k in range(len(addition_options)):
                if addition_options[k] == 0:
                    pass
                elif addition_options[k] == 1:
                    first_qubit_x += P2.x_exp[other_paulis,k+1+cq]
                elif addition_options[k] == 2:
                    first_qubit_x += P2.z_exp[other_paulis,k+1+cq]
                elif addition_options[k] == 3:
                    first_qubit_x += P2.x_exp[other_paulis,k+1+cq] + P2.z_exp[other_paulis,k+1+cq]
            if q_print:
                print('addition_options',addition_options)
                print('first_qubit_x',first_qubit_x%2)
                pass
            if not any(first_qubit_x%2):
                '''
                for j,k in enumerate(addition_options):
                    if addition_options_lens[j] > 1:
                        if k == 0:
                            pass
                        elif k == 1:
                            g = add_r2(P1,cq,j+1+cq)
                            P2 = g.act(P2)
                            for gg in g.gates:
                                C2.add_gate(gg)
                        elif k == 2:
                            g = add_s2(P1,cq,j+1+cq)
                            P2 = g.act(P2)
                            for gg in g.gates:
                                C2.add_gate(gg)
                        elif k == 3:
                            g = add_r2s2(P1,cq,j+1+cq)
                            P2 = g.act(P2) 
                            for gg in g.gates:
                                C2.add_gate(gg)
                '''
                break
    elif h_sym and not h_sym_impossible:
        addition_options = np.array([gate_opt[0] for gate_opt in gate_options])

    if q_print:
        print(addition_options)
        P3 = P2.copy()
        C3 = Circuit(dimensions=[2 for i in range(P1.n_qudits())])
        for j,k in enumerate(addition_options):
            if k == 0:
                g = add_phase(P1,cq,j+1+cq,'SSSS')
                for gg in g.gates:
                    C3.add_gate(gg)
                pass
            elif k == 1:
                g = add_r2(P1,cq,j+1+cq,'SSSS')
                #P2 = g.act(P2)
                for gg in g.gates:
                    C3.add_gate(gg)
            elif k == 2:
                g = add_s2(P1,cq,j+1+cq,'SSSS')
                #P2 = g.act(P2)
                for gg in g.gates:
                    C3.add_gate(gg)
            elif k == 3:
                g = add_r2s2(P1,cq,j+1+cq,'SSSS')
                #P2 = g.act(P2) 
                for gg in g.gates:
                    C3.add_gate(gg)
        P3 = C3.act(P3)
        print(P3)

    sym_pairs = set([])
    for i,spc_i in enumerate(sym_paulis_candidates):
        for j,spc_j in enumerate(sym_paulis_candidates[i+1:]):
            if abs(P2.weights[spc_i]) == abs(P2.weights[spc_j]) and np.array_equal(P2.x_exp[spc_i,current_qubit+1:],P2.x_exp[spc_j,current_qubit+1:]) and np.array_equal(P2.z_exp[spc_i,current_qubit+1:],P2.z_exp[spc_j,current_qubit+1:]):
                sym_pairs.add((spc_i,spc_j))
    if q_print:
        print('sym_pairs')
        print(sym_pairs)
    
    pairs_parity = []
    for pairs in sym_pairs:
        if abs(P2.weights[pairs[0]] * xi(P2.phases[pairs[0]],2) - P2.weights[pairs[1]] * xi(P2.phases[pairs[1]],2)) < 10**-10:
            pairs_parity.append(0)
        else:
            pairs_parity.append(1)
    if q_print:
        print('pairs_parity')
        print(pairs_parity)

    pairs_parity_copy = pairs_parity.copy()
    for n_gate in range(P1.n_qudits() - cq - 1): # iterate by number of gates, not 'random' extensive search
        if q_print:
            print('n_gate')
            print(n_gate)
        for comb in itertools.combinations(range(P1.n_qudits() - cq - 1), n_gate):
            if q_print:
                print('comb')
                print(comb)
            phase_changed_gates = np.zeros(P1.n_qudits() - cq - 1)
            for i in comb:
                phase_changed_gates[i] = 1
            
            phase_options_lens = []
            for l in phase_changed_gates:#
                if l == 0:
                    phase_options_lens.append(1)
                else:
                    phase_options_lens.append(8)
            if q_print:
                print('phase_options_lens')
                print(phase_options_lens)
            max_options = np.prod(phase_options_lens)
            for i in range(max_options):
                phase_options = int_to_bases(i,dims=phase_options_lens)
                if q_print:
                    print('phase_options')
                    print(phase_options)
                pairs_parity_copy = pairs_parity.copy()
                # update phase_parity based on that choice
                pairs_parity_copy = update_phase_parity(pairs_parity_copy,phase_options,P2,current_qubit,sym_pairs)
                if q_print:
                    print('pairs_parity_copy')
                    print(pairs_parity_copy)
                # If parity is correct, break out of loop
                if not any(pairs_parity_copy):
                    break
                else:
                    # check if any one gate option on one of the remaining qubits can fix the phases
                    remaining_qubits = [i + cq + 1  for i in range(P1.n_qudits() - cq - 1) if phase_changed_gates[i] == 0]
                    if q_print:
                        print('remaining_qubits')
                        print(remaining_qubits)
                    for q2 in remaining_qubits:
                        parity_dict = parity_difference(P2,cq,q2,sym_pairs,pairs_parity_copy)
                        if q_print:
                            print('parity_dict')
                            print(parity_dict)
                        parity_dict_lens = [len(parity_option) for parity_option in parity_dict.values()]
                        if q_print:
                            print('parity_dict_lens')
                            print(parity_dict_lens)
                        if 2 not in parity_dict_lens:
                            parity_dict_vals = []
                            for parity_option in parity_dict.values():
                                if 0 in parity_option:
                                    parity_dict_vals.append(0)
                                elif 1 in parity_option:
                                    parity_dict_vals.append(1)
                                else:
                                    parity_dict_vals.append(2)  
                            
                            if q_print:
                                print('parity_dict_vals')
                                print(parity_dict_vals)
                            
                            n_S = len([0 for _ in range(4) if parity_dict_vals[i] == 0])
                            n_D = len([1 for _ in range(4) if parity_dict_vals[i] == 1])
                            n_I = len([2 for _ in range(4) if parity_dict_vals[i] == 2])

                            if q_print:
                                print('n_S')
                                print(n_S)
                            if n_S != 3 and n_D != 3:
                                mode = transform_list(parity_dict_vals)
                                
                                if q_print:
                                    print('mode')
                                    print(mode)
                                if mode == 'SSSS':
                                    phase_options[q2 - cq - 1] = 0
                                elif mode == 'DDSS':
                                    phase_options[q2 - cq - 1] = 1
                                elif mode == 'DSDS':
                                    phase_options[q2 - cq - 1] = 2
                                elif mode == 'DSSD':
                                    phase_options[q2 - cq - 1] = 3
                                elif mode == 'SDDS':
                                    phase_options[q2 - cq - 1] = 4
                                elif mode == 'SDSD':
                                    phase_options[q2 - cq - 1] = 5
                                elif mode == 'SSDD':
                                    phase_options[q2 - cq - 1] = 6
                                elif mode == 'DDDD':
                                    phase_options[q2 - cq - 1] = 7 

                                pairs_parity_copy = update_phase_parity(pairs_parity.copy(),phase_options,P2,current_qubit,sym_pairs)
                                break
                    if not any(pairs_parity_copy):
                        break
            if not any(pairs_parity_copy):
                break
        if not any(pairs_parity_copy):
            break
    
    # at this point we have addition_options which gives us which type of gate to use and phase_options which gives us which phase mode to use
    # double check that the circuit and PauliSum are not double applied or something like that 
    if q_print:
        print('addition_options')
        print(addition_options)
        print('phase_options')
        print(phase_options)
    for j,k in enumerate(addition_options):
        if k == 0:
            g = add_phase(P1,cq,j+1+cq,mode_key(phase_options[j]))
            for gg in g.gates:
                C2.add_gate(gg)
            pass
        elif k == 1:
            g = add_r2(P1,cq,j+1+cq,mode_key(phase_options[j]))
            #P2 = g.act(P2)
            for gg in g.gates:
                C2.add_gate(gg)
        elif k == 2:
            g = add_s2(P1,cq,j+1+cq,mode_key(phase_options[j]))
            #P2 = g.act(P2)
            for gg in g.gates:
                C2.add_gate(gg)
        elif k == 3:
            g = add_r2s2(P1,cq,j+1+cq,mode_key(phase_options[j]))
            #P2 = g.act(P2) 
            for gg in g.gates:
                C2.add_gate(gg)
    if q_print:
        print('C2')
        print(C2)
    for gg in C2.gates:
        C.add_gate(gg)
    P2 = C2.act(P2)

    return(P2,C)


def PauliSum_Subspace(P,paulis,qubits):
    pauli_strings = P.pauli_strings
    weights = P.weights
    dimensions = P.dimensions
    phases = P.phases
    n_qudits = P.n_qudits()
    n_paulis = P.n_paulis()
    
    dimensions_new = np.array(dimensions)[qubits]
    pauli_strings_new = [PauliString(pauli_strings[i].x_exp[qubits],pauli_strings[i].z_exp[qubits],dimensions=dimensions_new) for i in paulis]
    weights_new = weights[paulis]
    phases_new = phases[paulis]
    P_new = PauliSum(pauli_strings_new, weights=weights_new ,dimensions=dimensions_new, phases=phases_new,standardise=False)
    return(P_new)

def add_uneven_phase(P1,q1,q2):
    C = Circuit(dimensions=[2 for i in range(P1.n_qudits())],gates = [CX(q2,q1,2),S(q1,2),CX(q2,q1,2),CX(q1,q2,2),S(q2,2),CX(q1,q2,2)])
    return(C)

def Find_Hadamard_Symmetries(P,q_print=False):
    P1 = P.copy()
    C = Circuit(dimensions=[2 for i in range(P1.n_qudits())])
    # construct a list of all possible PS combinations that have the same absolute value coefficient 
    cc = P1.weights
    cc_abs = np.abs(cc)
    cc_bands = group_indices(cc_abs)
    cc_bands = [np.array(b) for b in cc_bands if len(b) > 1]
    current_qubit = 0
    for ib,b in enumerate(cc_bands):
        for ic,pi in enumerate(b):
            for jc,pj in enumerate(b[ic+1:]):
                P_temp = PauliSum_Subspace(P1,[pi,pj],[i for i in range(current_qubit,P1.n_qudits())])
                if not P_temp.is_commuting():
                    P1,C,current_qubit = Find_Hadamard_Symmetries_iter_(P1,C,pi,pj,current_qubit,cc_bands)
                    if q_print:
                        print(pi,pj)
                        print(P1)
                        print()
    return(P1,C)

def Find_Hadamard_Symmetries_iter_(P1,C,pi,pj,current_qubit,cc_bands,q_print=False):
    C1 = Circuit(dimensions=[2 for i in range(P1.n_qudits())])
    #for g in C.gates:
    #    C1.add_gate(g)
    
    P2 = P1.copy()
    C1, P2, px, pz = prepare_sym_candidates(P2,pi,pj,C1,current_qubit)
    if q_print:
        print('Original')
        print(P2)
        print()
        print('Circuit')
        print(C1.act(P1.copy()))
    

    # cancel for pauli with x
    P2, C1 = cancel_pauli(P2, current_qubit, px, C1, P2.n_qudits())
    # cancel for pauli with z
    g = H(current_qubit, P.dimensions[current_qubit])
    C1.add_gate(g)
    P2 = g.act(P2)
    P2, C1 = cancel_pauli(P2, current_qubit, pz, C1, P2.n_qudits())  
    g = H(current_qubit, P.dimensions[current_qubit])
    C1.add_gate(g)
    P2 = g.act(P2)
    if q_print:
        print('Make X111... and Z111...')
        print(P2)
        #print(C1.act(P1.copy()))


    current_qubit_pauli_check = check_current_paulis(P2,pi,pj,current_qubit,cc_bands)
    #print('current_qubit_pauli_check',current_qubit_pauli_check)
    if not current_qubit_pauli_check:
        # not a candidate for Hadamard symmetry, return just the original P1 and C
        return(P1,C,current_qubit)

    for i in range(current_qubit+1,P1.n_qudits()):
        #print(i)
        C1, pivots = symplectic_reduction_iter_qudit_(P1.copy(), C1, [], i)
        #print(C1.act(P1.copy()))

    P2 = C1.act(P1.copy())
    if q_print:
        print(P2)

    # cancel the Y's in the first qubit
    #print(P2)
    P2,C1 = remove_Ys(P2,C1,current_qubit,q_print=q_print)
    if q_print:    
        print(P2)

    # check if all Y's in the first qubit have been cancelled
    sym_paulis_candidates = [i for i in range(P2.n_paulis()) if (P2.x_exp[i,current_qubit] == 1 and P2.z_exp[i,current_qubit] == 0) 
    or (P2.x_exp[i,current_qubit] == 0 and P2.z_exp[i,current_qubit] == 1)]
    other_paulis = [i for i in range(P2.n_paulis()) if i not in sym_paulis_candidates]
    first_qubit_x = P2.x_exp[other_paulis,current_qubit]
    if any(first_qubit_x%2):
        #print('No Hadamard Symmetry')
        return(P1,C,current_qubit)
    
    # deal with phases
    cq = current_qubit
    
    sym_pairs = set([])
    for i,spc_i in enumerate(sym_paulis_candidates):
        for j,spc_j in enumerate(sym_paulis_candidates[i+1:]):
            if abs(P2.weights[spc_i]) == abs(P2.weights[spc_j]) and np.array_equal(P2.x_exp[spc_i,current_qubit+1:],P2.x_exp[spc_j,current_qubit+1:]) and np.array_equal(P2.z_exp[spc_i,current_qubit+1:],P2.z_exp[spc_j,current_qubit+1:]):
                sym_pairs.add((spc_i,spc_j))
    #print(sym_pairs)
    
    pairs_parity = []
    for pairs in sym_pairs:
        if abs(P2.weights[pairs[0]] * xi(P2.phases[pairs[0]],2) - P2.weights[pairs[1]] * xi(P2.phases[pairs[1]],2)) < 10**-10:
            pairs_parity.append(0)
        else:
            pairs_parity.append(1)
    #print(pairs_parity)
    max_options = 8**(P.n_qudits() - current_qubit - 1)
    for i in range(max_options):
        phase_options = int_to_bases(i,dims=[8]*(P.n_qudits() - current_qubit - 1))
        #print(phase_options)
        pairs_parity_copy = pairs_parity.copy()
        for k in range(len(phase_options)):
            #print(k)
            if phase_options[k] == 0: # SSSS
                pass
            elif phase_options[k] == 1: # DDSS
                for ip,pairs in enumerate(sym_pairs):
                    if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        pass
                #
            elif phase_options[k] == 2: # DSDS
                for ip,pairs in enumerate(sym_pairs):
                    if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        pass
                #
            elif phase_options[k] == 3: # DSSD
                for ip,pairs in enumerate(sym_pairs):
                    if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                #
            elif phase_options[k] == 4: # SDDS
                for ip,pairs in enumerate(sym_pairs):
                    if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        pass
                #
            elif phase_options[k] == 5: # SDSD
                for ip,pairs in enumerate(sym_pairs):
                    if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                #
            elif phase_options[k] == 6: # SSDD
                for ip,pairs in enumerate(sym_pairs):
                    if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                #
            elif phase_options[k] == 7: # DDDD
                for ip,pairs in enumerate(sym_pairs):
                    if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
                    elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                        pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                        #pass
        if q_print:
            print(pairs_parity_copy)
        if not any(pairs_parity_copy):
            for j,k in enumerate(phase_options):
                if k == 0:
                    #print(j,k)
                    #print('Identity',qi)
                    #print(P2)
                    #print()
                    pass
                elif k == 1:
                    #print(j,k)
                    g = add_phase(P1,cq,j+1+cq,'DDSS')
                    P2 = g.act(P2)
                    for gg in g.gates:
                        C1.add_gate(gg)
                    #print(P2)
                    #print()
                elif k == 2:
                    #print(j,k)
                    g = add_phase(P1,cq,j+1+cq,'DSDS')
                    P2 = g.act(P2)
                    for gg in g.gates:
                        C1.add_gate(gg)
                    #print(P2)
                    #print()
                elif k == 3:
                    g = add_phase(P1,cq,j+1+cq,'DSSD')
                    P2 = g.act(P2) 
                    for gg in g.gates:
                        C1.add_gate(gg)
                elif k == 4:
                    g = add_phase(P1,cq,j+1+cq,'SDDS')
                    P2 = g.act(P2) 
                    for gg in g.gates:
                        C1.add_gate(gg)
                elif k == 5:
                    g = add_phase(P1,cq,j+1+cq,'SDSD')
                    P2 = g.act(P2) 
                    for gg in g.gates:
                        C1.add_gate(gg)
                elif k == 6:
                    g = add_phase(P1,cq,j+1+cq,'SSDD')
                    P2 = g.act(P2) 
                    for gg in g.gates:
                        C1.add_gate(gg)
                elif k == 7:
                    g = add_phase(P1,cq,j+1+cq,'DDDD')
                    P2 = g.act(P2) 
                    for gg in g.gates:
                        C1.add_gate(gg)
            break
            
    # ToDo: final check for Hadamard symmetry
    # only x and z in the first qubit if both have the same coefficient
    # no problematic phases
    # after the x and z, all paulis are the same


    C2 = Circuit(dimensions=[2 for i in range(P1.n_qudits())])
    for g in C.gates:
        C2.add_gate(g)
    for g in C1.gates:
        C2.add_gate(g)

    return(P2,C2,current_qubit+1)
    

In [41]:
P,C = Hadamard_Symmetric_PauliSum(25,8,4)
print(P)

[0.23209873]
[ 0.31275128 -0.93151414]
[0.65886289]
[-0.44160693]
[12, 7, 19, 5, 21] [13, 8, 18, 4, 20]
(1.4520211438634234-0j)  |x1z0 x0z1 x0z0 x1z1 x1z0 x0z1 x1z0 x0z1 | 0 
(-1.8086472399529443+0j) |x0z1 x1z1 x0z1 x0z1 x1z0 x1z1 x1z1 x0z1 | 0 
(-0.7456359581417605+0j) |x0z0 x1z1 x0z0 x0z0 x1z0 x1z0 x1z1 x0z1 | 0 
(0.6588628948310071+0j)  |x1z1 x0z0 x0z0 x0z1 x1z0 x0z1 x0z1 x1z1 | 0 
(0.6588628948310071+0j)  |x1z0 x1z1 x1z1 x0z1 x1z1 x0z0 x1z0 x1z0 | 0 
(-0.6588628948310071+0j) |x0z0 x0z1 x1z0 x1z0 x0z0 x0z1 x0z0 x1z0 | 0 
(1.8837924330772964+0j)  |x0z0 x1z0 x0z1 x1z1 x1z0 x1z1 x0z0 x1z1 | 0 
(0.3127512825520639+0j)  |x1z0 x1z1 x0z1 x1z0 x0z0 x1z0 x1z1 x1z0 | 0 
(-0.3127512825520639+0j) |x0z1 x1z1 x0z0 x1z0 x0z0 x0z1 x0z1 x1z0 | 0 
(0.6876113474071103-0j)  |x0z1 x1z0 x0z0 x0z0 x0z1 x0z1 x0z0 x0z0 | 0 
(0.6876113474071103-0j)  |x1z1 x1z1 x0z1 x0z0 x0z0 x0z0 x1z0 x1z0 | 0 
(0.6445998731844786-0j)  |x1z0 x1z0 x0z0 x0z1 x1z1 x0z0 x0z1 x1z1 | 0 
(-0.23209872542104537+0j)|x0z0 x0z1 x0z0 x0z

In [ ]:
pauli_sum,C = Find_Hadamard_Symmetries(P)
print(pauli_sum)

single Pauli forcing
[[0], [1], [2], [0, 1, 2, 3], [0, 1, 2, 3], [0, 1, 2, 3], [0, 1, 2, 3]]
3
[1. 1.]
[1. 1.]
[0. 0.]
[0. 0.]
single Pauli forcing
[[0], [1], [2], [2, 3], [0, 1, 2, 3], [0, 1, 2, 3], [0, 1, 2, 3]]
4
[0.]
[0.]
[1.]
[1.]
(-0.14769847458841456+0j)|x0z0 x1z0 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0 | 0 
(-0.14769847458841456+0j)|x0z0 x0z1 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0 | 1 
(0.20405586279402807+0j) |x0z0 x1z1 x0z1 x0z0 x0z0 x0z0 x0z0 x0z0 | 0 
(-0.20405586279402807+0j)|x0z1 x0z0 x0z1 x0z1 x1z0 x0z0 x1z0 x0z0 | 0 
(0.20405586279402807+0j) |x1z0 x0z0 x0z1 x0z1 x1z0 x0z0 x1z0 x0z0 | 1 
(1.2668798717104328-0j)  |x0z0 x0z0 x1z0 x1z0 x0z0 x0z0 x0z0 x0z0 | 0 
(-0.3024591251832312+0j) |x0z0 x1z1 x0z0 x1z1 x0z1 x0z0 x0z0 x0z0 | 1 
(1.358875089434938-0j)   |x1z0 x1z1 x0z1 x1z1 x0z0 x0z0 x1z0 x0z0 | 1 
(1.358875089434938-0j)   |x0z1 x1z1 x0z1 x1z1 x0z0 x0z0 x1z0 x0z0 | 1 
(-3.356056276303437+0j)  |x0z0 x1z1 x1z0 x0z1 x0z1 x0z0 x0z0 x0z0 | 0 
(-1.3307380703280127+0j) |x0z0 x1z1 x1z0 x0z0 x0z0 x0z

In [22]:
cc = P.weights
cc_abs = np.abs(cc)
cc_bands = group_indices(cc_abs)
cc_bands = [np.array(b) for b in cc_bands if len(b) > 1]
C = Circuit(dimensions=[2 for i in range(P.n_qudits())])
print(P)
print()
pauli_sum,C,current_qubit = Find_Hadamard_Symmetries_iter_(P,C,0,1,0,cc_bands,q_print=True)
#print(current_qubit)
#P1,C,current_qubit = Find_Hadamard_Symmetries_iter_(P1,C,11,12,current_qubit,cc_bands,q_print=False)
#P1,C,current_qubit = Find_Hadamard_Symmetries_iter_(P1,C,23,24,current_qubit,cc_bands,q_print=True)
#P1,C = Find_Hadamard_Symmetries(P)
print(pauli_sum)
print(pauli_sum == C.act(P))

(-1.3294515464331034+0j) |x1z0 x1z0 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0 | 1 
(0.5077572177050729+0j)  |x0z1 x1z0 x0z1 x0z0 x0z0 x0z0 x0z0 x0z0 | 1 
(-0.9102986294267842+0j) |x1z0 x0z1 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0 | 1 
(0.9102986294267842+0j)  |x1z1 x1z0 x1z0 x0z0 x0z0 x0z0 x0z0 x0z0 | 0 
(-0.9102986294267842+0j) |x1z1 x1z0 x1z1 x1z0 x0z0 x0z0 x0z0 x0z0 | 1 
(0.9102986294267842+0j)  |x1z1 x0z0 x1z0 x0z1 x0z0 x0z0 x0z0 x0z0 | 0 
(-1.1421161219148122+0j) |x1z0 x1z1 x1z1 x1z1 x0z0 x0z0 x0z0 x0z1 | 0 
(1.1421161219148122+0j)  |x1z1 x1z0 x0z0 x1z0 x1z0 x0z0 x0z0 x0z0 | 0 
(-1.1421161219148122+0j) |x1z0 x1z1 x1z0 x1z1 x1z0 x0z0 x0z0 x0z0 | 1 
(-0.6515606287459905+0j) |x1z1 x1z0 x0z0 x0z0 x0z1 x0z0 x0z0 x0z0 | 1 
(-0.4977811866941862+0j) |x1z0 x0z0 x1z0 x1z1 x0z0 x1z0 x0z0 x0z0 | 0 
(-0.18419823797018203+0j)|x1z1 x0z0 x1z0 x1z1 x0z1 x0z0 x0z1 x0z0 | 1 
(0.18419823797018203+0j) |x1z0 x1z1 x0z0 x0z0 x0z0 x0z1 x0z0 x0z0 | 1 
(0.18419823797018203+0j) |x0z1 x1z0 x1z0 x1z1 x0z1 x1z0 x0z0 x0z0 | 0 
(0.222

In [44]:
cc = P.weights
cc_abs = np.abs(cc)
cc_bands = group_indices(cc_abs)
cc_bands = [np.array(b) for b in cc_bands if len(b) > 1]
C = Circuit(dimensions=[2 for i in range(P.n_qudits())])
print(P)
print()
pi = 7
pj = 8
current_qubit = 0

C1 = Circuit(dimensions=[2 for i in range(P.n_qudits())])
P1 = P.copy()
C1, P1, px, pz = prepare_sym_candidates(P1,pi,pj,C1,current_qubit)

#print('Original')
#print(P2)
#print()
#print('Circuit')
#print(C1.act(P1.copy()))
#print()

# cancel for pauli with x
P1, C1 = cancel_pauli(P1, current_qubit, px, C1, P1.n_qudits())
# cancel for pauli with z
g = H(current_qubit, P1.dimensions[current_qubit])
C1.add_gate(g)
P1 = g.act(P1)
P1, C1 = cancel_pauli(P1, current_qubit, pz, C1, P1.n_qudits())  
g = H(current_qubit, P1.dimensions[current_qubit])
C1.add_gate(g)
P1 = g.act(P1)

#print('Make X111... and Z111...')
#print(P2)
#print()
#print(C1.act(P1.copy()))


current_qubit_pauli_check = check_current_paulis(P1,pi,pj,current_qubit,cc_bands)
#print('current_qubit_pauli_check',current_qubit_pauli_check)
if not current_qubit_pauli_check:
    # not a candidate for Hadamard symmetry, return just the original P1 and C
    print('Current_qubit_pauli_check failed')

for i in range(current_qubit+1,P.n_qudits()):
    #print(i)
    C1, pivots = symplectic_reduction_iter_qudit_(P.copy(), C1, [], i)
    #print(C1.act(P1.copy()))

P1 = C1.act(P.copy())
print('Position to remove Ys')
print(P1)

'''
# cancel the Y's in the first qubit
#print(P2)
cq = current_qubit
sym_paulis_candidates,other_paulis = hadamard_symmetry_pauli_candidates(P1,current_qubit)

# find forcings
gate_options = find_forced_gates_to_remove_Ys(P1,cq,sym_paulis_candidates,other_paulis)
max_gate_options = max([len(ig) for ig in gate_options])
min_gate_options = min([len(ig) for ig in gate_options])
print(gate_options)

if min_gate_options == 0:
    print('Problem')
if max_gate_options > 1:
    addition_options = search_gate_options(P1,cq,gate_options,other_paulis)
else:
    addition_options = np.array([g[0] for g in gate_options])
print(addition_options)

phase_option = find_phase_option(P1,cq)
print(phase_option)

C2 = get_remove_Y_circuit(P1,cq,addition_options,phase_option)
P1 = C2.act(P1)
for g in C2.gates:
    C1.add_gate(g)
'''
P1,C1 = remove_Ys(P1,C1,current_qubit,q_print = False)
print('Ys removed?')
print(P1)

(1.4520211438634234-0j)  |x1z0 x0z1 x0z0 x1z1 x1z0 x0z1 x1z0 x0z1 | 0 
(-1.8086472399529443+0j) |x0z1 x1z1 x0z1 x0z1 x1z0 x1z1 x1z1 x0z1 | 0 
(-0.7456359581417605+0j) |x0z0 x1z1 x0z0 x0z0 x1z0 x1z0 x1z1 x0z1 | 0 
(0.6588628948310071+0j)  |x1z1 x0z0 x0z0 x0z1 x1z0 x0z1 x0z1 x1z1 | 0 
(0.6588628948310071+0j)  |x1z0 x1z1 x1z1 x0z1 x1z1 x0z0 x1z0 x1z0 | 0 
(-0.6588628948310071+0j) |x0z0 x0z1 x1z0 x1z0 x0z0 x0z1 x0z0 x1z0 | 0 
(1.8837924330772964+0j)  |x0z0 x1z0 x0z1 x1z1 x1z0 x1z1 x0z0 x1z1 | 0 
(0.3127512825520639+0j)  |x1z0 x1z1 x0z1 x1z0 x0z0 x1z0 x1z1 x1z0 | 0 
(-0.3127512825520639+0j) |x0z1 x1z1 x0z0 x1z0 x0z0 x0z1 x0z1 x1z0 | 0 
(0.6876113474071103-0j)  |x0z1 x1z0 x0z0 x0z0 x0z1 x0z1 x0z0 x0z0 | 0 
(0.6876113474071103-0j)  |x1z1 x1z1 x0z1 x0z0 x0z0 x0z0 x1z0 x1z0 | 0 
(0.6445998731844786-0j)  |x1z0 x1z0 x0z0 x0z1 x1z1 x0z0 x0z1 x1z1 | 0 
(-0.23209872542104537+0j)|x0z0 x0z1 x0z0 x0z1 x0z0 x0z0 x1z0 x1z1 | 0 
(-0.23209872542104537+0j)|x0z0 x1z1 x0z0 x1z1 x1z0 x1z0 x1z1 x1z1 | 0 
(-0.39

In [37]:
# makes a list of the paulis that currently have x and z in the first qubit as well as a list of all remaining paulis
def hadamard_symmetry_pauli_candidates(pauli_sum,current_qubit):
    sym_paulis_candidates = [i for i in range(pauli_sum.n_paulis()) if (pauli_sum.x_exp[i,current_qubit] == 1 and pauli_sum.z_exp[i,current_qubit] == 0) or (pauli_sum.x_exp[i,current_qubit] == 0 and pauli_sum.z_exp[i,current_qubit] == 1)]
    other_paulis = [i for i in range(pauli_sum.n_paulis()) if i not in sym_paulis_candidates]
    return(sym_paulis_candidates,other_paulis)

def reduce_gate_options(gate_options,gates_to_remove,qi,cq):
    for i in gates_to_remove:
        if i in gate_options[qi-(1+cq)]:
            gate_options[qi-(1+cq)].remove(i)
    return(gate_options)

def forcing_options(P,usable_qubits,sym_paulis_candidates):
    # For each PS, how many identities does it have on the still 'usable_qubits'
    list_n_I = [number_of_I(P,ip,usable_qubits) for ip in range(P.n_paulis())]
    
    # Locate Paulis that have all Identities (except for one) in the usable qubits and are not candidates for symmetry
    forcing_paulis = [i for i in range(P.n_paulis()) if list_n_I[i] == len(usable_qubits) - 1 and i not in sym_paulis_candidates]
    
    # locate the qubits on which the individual non-I paulis are
    forcing_qubits = []
    for i in forcing_paulis:
        qubit_index = min([j for j in usable_qubits if (P.x_exp[i,j]+P.z_exp[i,j]) != 0])
        if qubit_index not in forcing_qubits:
            forcing_qubits.append(qubit_index)
    
    return(forcing_paulis,forcing_qubits)

def find_forced_gates_to_remove_Ys(P,cq,sym_paulis_candidates,other_paulis):

    h_sym = False # variable that becomes true once the PS is hadamard symmetric (afterwards we don't need to search for more forcings)
    no_remaining_forcings = False # variable that becomes true if there are no remaining forcings

    usable_qubits = [i for i in range(cq+1,P.n_qudits())]
    gate_options = [[0,1,2,3] for i in range(cq+1,P.n_qudits())]

    # x-component of the first qubit for all paulis (object that will be changed via forcings)
    all_first_qubit = P.x_exp[:,cq]

    while not no_remaining_forcings and len(usable_qubits) > 0 and not h_sym:
        forcing_paulis,forcing_qubits = forcing_options(P,usable_qubits,sym_paulis_candidates)

        no_remaining_forcings = True
        for qi in forcing_qubits:
            pauli_indexes = [i for i in forcing_paulis if (P.x_exp[i,qi]+P.z_exp[i,qi]) != 0]
            pauli_indexes_first_qubit = np.copy(all_first_qubit[pauli_indexes])
            
            # determine the number of different non-I paulis are on qubit qi for pauli_indexes (turn into own function)
            forcing_pauli_types = set([])
            for i in pauli_indexes:
                if P.x_exp[i,qi] == 1 and P.z_exp[i,qi] == 0:
                    forcing_pauli_types.add(1)
                elif P.x_exp[i,qi] == 0 and P.z_exp[i,qi] == 1:
                    forcing_pauli_types.add(2)
                elif P.x_exp[i,qi] == 1 and P.z_exp[i,qi] == 1:
                    forcing_pauli_types.add(3)

            if len(forcing_pauli_types) > 1:
                if not any(pauli_indexes_first_qubit):
                    # Identity
                    gate_options = reduce_gate_options(gate_options,[1,2,3],qi,cq)

                elif not any((pauli_indexes_first_qubit + P.x_exp[pauli_indexes,qi])%2):
                    # add_r2
                    gate_options = reduce_gate_options(gate_options,[0,2,3],qi,cq)
                    all_first_qubit = (all_first_qubit + P.x_exp[:,qi])%2
                    other_first_qubit = all_first_qubit[other_paulis]

                elif not any((pauli_indexes_first_qubit + P.z_exp[pauli_indexes,qi])%2):
                    # add_s2
                    gate_options = reduce_gate_options(gate_options,[0,1,3],qi,cq)
                    all_first_qubit = (all_first_qubit + P.z_exp[:,qi])%2

                elif not any((pauli_indexes_first_qubit + P.z_exp[pauli_indexes,qi] + P.x_exp[pauli_indexes,qi])%2):
                    # add_r2s2
                    gate_options = reduce_gate_options(gate_options,[0,1,2],qi,cq)
                    all_first_qubit = (all_first_qubit + P.x_exp[:,qi] + P.z_exp[:,qi])%2
                else:
                    # Forcing Impossible
                    gate_options = reduce_gate_options(gate_options,[0,1,2,3],qi,cq)
                    return(gate_options)
                no_remaining_forcings = False
                usable_qubits.remove(qi)
            else:
                removable_gates = []
                if any(pauli_indexes_first_qubit):
                    removable_gates.append(0)
                if any((pauli_indexes_first_qubit + P.x_exp[pauli_indexes,qi])%2):
                    removable_gates.append(1)
                if any((pauli_indexes_first_qubit + P.z_exp[pauli_indexes,qi])%2):
                    removable_gates.append(2)
                if any((pauli_indexes_first_qubit + P.z_exp[pauli_indexes,qi] + P.x_exp[pauli_indexes,qi])%2):
                    removable_gates.append(3) 
                gate_options = reduce_gate_options(gate_options,removable_gates,qi,cq)
            
            other_first_qubit = all_first_qubit[other_paulis]
            if not any(other_first_qubit%2):
                h_sym = True
                for iq in usable_qubits:
                    gate_options = reduce_gate_options(gate_options,[1,2,3],iq,cq)
    
    return(gate_options)

def search_gate_options(P,cq,gate_options,other_paulis):
    addition_options_lens = []
    for g in gate_options:
        addition_options_lens.append(len(g))
    max_options = np.prod(addition_options_lens)
    for i in range(max_options):
        addition_options_temp = int_to_bases(i,dims=addition_options_lens[::-1])[::-1]
        addition_options = np.array([gate_options[j][addition_options_temp[j]] for j in range(P.n_qudits() - cq - 1)])
        first_qubit = P.x_exp[other_paulis,cq]
        for k in range(P.n_qudits() - cq - 1):
            if addition_options[k] == 0:
                pass
            elif addition_options[k] == 1:
                first_qubit += P.x_exp[other_paulis,k+1+cq]
            elif addition_options[k] == 2:
                first_qubit += P.z_exp[other_paulis,k+1+cq]
            elif addition_options[k] == 3:
                first_qubit += P.x_exp[other_paulis,k+1+cq] + P.z_exp[other_paulis,k+1+cq]
        if not any(first_qubit%2):
            return(addition_options)
    # ToDo: Find a way to handle the case where no way to remove all Ys can be found

def find_hadamard_sym_pairs(P,current_qubit,sym_paulis_candidates):
    sym_pairs = set([])
    for i,spc_i in enumerate(sym_paulis_candidates):
        for j,spc_j in enumerate(sym_paulis_candidates[i+1:]):
            if abs(P.weights[spc_i]) == abs(P.weights[spc_j]) and np.array_equal(P.x_exp[spc_i,current_qubit+1:],P.x_exp[spc_j,current_qubit+1:]) and np.array_equal(P.z_exp[spc_i,current_qubit+1:],P.z_exp[spc_j,current_qubit+1:]):
                sym_pairs.add((spc_i,spc_j))
    return(sym_pairs)

def calculate_pairs_parity(P,sym_pairs):
    pairs_parity = []
    for pairs in sym_pairs:
        if abs(P.weights[pairs[0]] * xi(P.phases[pairs[0]],2) - P.weights[pairs[1]] * xi(P.phases[pairs[1]],2)) < 10**-10:
            pairs_parity.append(0)
        else:
            pairs_parity.append(1)
    return(pairs_parity)

def mode_to_int(mode):
    if mode == 'SSSS':
        return(0)
    elif mode == 'DDSS':
        return(1)
    elif mode == 'DSDS':
        return(2)
    elif mode == 'DSSD':
        return(3)
    elif mode == 'SDDS':
        return(4)
    elif mode == 'SDSD':
        return(5)
    elif mode == 'SSDD':
        return(6)
    elif mode == 'DDDD':
        return(7)
    else:
        raise ValueError("Mode must be one of SSSS, DDSS, DSDS, DSSD, SDDS, SDSD, SSDD, DDDD")
    

def find_phase_option(P,cq):
    sym_pairs = find_hadamard_sym_pairs(P,current_qubit,sym_paulis_candidates)
    pairs_parity = calculate_pairs_parity(P,sym_pairs)
    for n_gate in range(P.n_qudits() - cq - 1): # iterate by number of gates, not 'random' extensive search
        for comb in itertools.combinations(range(P.n_qudits() - cq - 1), n_gate): # choose n_gate gates to have a phase
            phase_options_lens = np.zeros(P.n_qudits() - cq - 1) + 1
            for i in comb:
                phase_options_lens[i] = 8
            max_options = int(np.prod(phase_options_lens))
            for i in range(max_options):
                pairs_parity_copy = pairs_parity.copy()
                phase_options = int_to_bases(i,dims=phase_options_lens)
                pairs_parity_copy = update_phase_parity(pairs_parity_copy,phase_options,P,current_qubit,sym_pairs)
                if not any(pairs_parity_copy):
                    return(phase_options)
                else:
                    # check if any one gate option on one of the remaining qubits can fix the phases
                    remaining_qubits = [i + cq + 1  for i in range(P.n_qudits() - cq - 1) if phase_options_lens[i] == 1]
                    for q2 in remaining_qubits:
                        parity_dict = parity_difference(P,cq,q2,sym_pairs,pairs_parity_copy)
                        parity_dict_lens = [len(parity_option) for parity_option in parity_dict.values()]
                        if 2 not in parity_dict_lens:
                            parity_dict_vals = []
                            for parity_option in parity_dict.values():
                                if 0 in parity_option:
                                    parity_dict_vals.append(0)
                                elif 1 in parity_option:
                                    parity_dict_vals.append(1)
                                else:
                                    parity_dict_vals.append(2)
                            n_S = len([0 for _ in range(4) if parity_dict_vals[i] == 0])
                            n_D = len([1 for _ in range(4) if parity_dict_vals[i] == 1])
                            if n_S != 3 and n_D != 3:
                                mode = transform_list(parity_dict_vals)
                                phase_options[q2 - cq - 1] = mode_to_int(mode)
                                pairs_parity_copy = update_phase_parity(pairs_parity.copy(),phase_options,P,current_qubit,sym_pairs)
                                if not any(pairs_parity_copy):
                                    return(phase_options)
                                else:
                                    print('Weird')
                
    # ToDo: Find a way to handle the case where no way to remove all Phases can be found
    return(phase_options)

def add_gate(i):
    if i == 0:
        return(add_phase)
    elif i == 1:
        return(add_r2)
    elif i == 2:
        return(add_s2)
    elif i == 3:
        return(add_r2s2)
    
def int_to_mode(n):
    if n == 0:
        return('SSSS')
    elif n == 1:
        return('DDSS')
    elif n == 2:
        return('DSDS')
    elif n == 3:
        return('DSSD')
    elif n == 4:
        return('SDDS')
    elif n == 5:
        return('SDSD')
    elif n == 6:
        return('SSDD')
    elif n == 7:
        return('DDDD')
    else:
        raise ValueError('Value has to be between 0 and 7')

def get_remove_Y_circuit(P,cq,addition_options,phase_options):
    C = Circuit(dimensions=[2 for i in range(P.n_qudits())])
    for i,g in enumerate(addition_options):
        c = add_gate(g)(P,cq,i+1+cq,int_to_mode(phase_options[i]))
        for ig in c.gates:
            C.add_gate(ig)
    return(C)
    

In [43]:
def remove_Ys(P1,C,current_qubit,q_print = False):
    cq = current_qubit
    sym_paulis_candidates,other_paulis = hadamard_symmetry_pauli_candidates(P1,current_qubit)

    # find forcings
    gate_options = find_forced_gates_to_remove_Ys(P1,cq,sym_paulis_candidates,other_paulis)
    max_gate_options = max([len(ig) for ig in gate_options])
    min_gate_options = min([len(ig) for ig in gate_options])
    
    # search through remaining gate options
    if min_gate_options == 0:
        return(P1,C)
    if max_gate_options > 1:
        addition_options = search_gate_options(P,cq,gate_options,other_paulis)
    else:
        addition_options = np.array([g[0] for g in gate_options])

    # find phase options
    phase_options = find_phase_option(P1,cq)
    
    # at this point we have addition_options which gives us which type of gate to use and phase_options which gives us which phase mode to use
    C1 = get_remove_Y_circuit(P1,cq,addition_options,phase_options)
    P1 = C1.act(P1)
    for g in C1.gates:
        C.add_gate(g)

    return(P1,C)